In [1]:
from transformers import AutoTokenizer, T5ForConditionalGeneration
import torch
import pandas as pd

[2023-07-24 09:59:16,286] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


2023-07-24 09:59:22.414409: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-24 09:59:22.864785: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-24 09:59:22.927217: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-24 09:59:25.423355: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:
model_name = "T5-Finetuned-with-IMDB-Spider"

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
device

'cuda'

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)
model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [4]:
from datasets import Dataset

df = pd.read_csv('Training_set_IMDB/testing_set_no_target.csv')
df = df.sample(frac =1).reset_index(drop=True)
for index,row in df.iterrows():
    df.loc[index, 'Text'] = "Translate to SQL: " + row['Text']


test_set_seen = Dataset.from_pandas(df)

In [5]:
test_set_seen.set_format(type = "torch")

print(test_set_seen["SQL"][1])
print(test_set_seen["Text"][1])

select rank from movies t1 where t1.name = 'Misleading Lady, The'
Translate to SQL: what is the movie 'Misleading Lady, The' rank


In [6]:
def convert_to_features(example_batch, padding = "max_length",input_max = 256, output_max = 256):
    inputs = tokenizer.batch_encode_plus(example_batch["Text"], max_length=input_max, is_split_into_words = False, padding='max_length', truncation=True, return_tensors = "pt")
    
    targets = tokenizer.batch_encode_plus(example_batch["SQL"], max_length=output_max, padding = "max_length",truncation = True)
    if padding == "max_length":
        targets["inputs_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in target] for target in targets["input_ids"]
        ]
    
    inputs["labels"] = targets['input_ids']
    return inputs

def evaluate_peft_model(sample):
    outputs = model.generate(input_ids=sample["input_ids"].unsqueeze(0).cuda(), max_length = 200, top_p=0.9)
    prediction = tokenizer.decode(outputs[0].detach().cpu().numpy(), skip_special_tokens=True)
    label = np.where(sample['labels'] != -100, sample['labels'], tokenizer.pad_token_id)
    label = tokenizer.decode(label, skip_special_tokens=True)
    _ = execution_accuracy(prediction, label)
    return prediction, label

def execution_accuracy(prediction, label):
    try:
        
        cursor.execute(label)
        result_label = cursor.fetchall()
        all_executions_overall.append(1)
        try:
            cursor.execute(prediction)
            result_pred = cursor.fetchall()
            all_executions_accuracy.append(1)
            if len(result_label)>10:
                if len(result_label) == len(result_pred):
                    accurate_executions.append(1) 
            elif result_label == result_pred:
                accurate_executions.append(1)
            else:
                for_checking_label.append(label)
                for_checking_prediction.append(prediction)
                
        except:
            failed_executions.append(1)
            failed_predicted_SQL.append(prediction)
                
    except:
        failed_original_SQL.append(label)
    return None



In [7]:
import evaluate
import numpy as np
from tqdm import tqdm
import mysql.connector



connection = mysql.connector.connect(
    host="relational.fit.cvut.cz",
    user="guest",
    password="relational",
    database="imdb_ijs"
)
cursor = connection.cursor()



print("mapping both datasets")
tokenized_dataset = test_set_seen.map(convert_to_features, batched=True, num_proc=4)

print("mapped both dataset")
print("Document we have: tokenized_dataset for seen data")


print("\n\n Running executions for seen dataset")
all_executions_overall = []
failed_executions = []
all_executions_accuracy = []
accurate_executions = []
for_checking_label = []
for_checking_prediction = []
failed_original_SQL = []
failed_predicted_SQL = []



for sample in tqdm(tokenized_dataset):
    p,l = evaluate_peft_model(sample)



print("All SQL runs: ", len(all_executions_overall))
print("Model SQLs that failed: ", len(failed_executions))
print(f"Execution rate: {len(all_executions_accuracy)/len(all_executions_overall)*100}%")
print(f"Execution rate: {100 - len(failed_executions)/len(all_executions_overall)*100}%")
print(f"Execution accuracy: {len(accurate_executions)/len(all_executions_accuracy)*100}%")

# failed_original_sql_df = pd.DataFrame(failed_original_SQL)
# failed_predicted_sql_df = pd.DataFrame(failed_predicted_SQL)
# not_equals = pd.DataFrame({
#     'Label':for_checking_label,
#     'Prediction': for_checking_prediction
# })

#not_equals.to_csv("/home/toibazd/Data/Text2SQL/Training_set_IMDB/Not_equals_Spider.csv", index = False)
#failed_original_sql_df.to_csv("/home/toibazd/Data/Text2SQL/Training_set_IMDB/Failed_originals_Spider.csv", index = False)
#failed_predicted_sql_df.to_csv("/home/toibazd/Data/Text2SQL/Training_set_IMDB/Failed_predicted_Spider.csv", index = False)

mapping both datasets


Map (num_proc=4):   0%|          | 0/2315 [00:00<?, ? examples/s]

mapped both dataset
Document we have: tokenized_dataset for seen data


 Running executions for seen dataset


100%|██████████| 2315/2315 [40:33<00:00,  1.05s/it]

All SQL runs:  2254
Model SQLs that failed:  162
Execution rate: 92.81277728482698%
Execution rate: 92.81277728482698%
Execution accuracy: 94.16826003824092%
